### The recommender system is based on KNN algorithm, where similar films are recommended

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
movie = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
movie.info()
movie.head()

In [ ]:
# Finding no. of movies for each genre
genre= {}
for item in movie['genres']:
    genres = item.split('|')
    for i in genres:
        genre[i] = genre.get(i, 0) + 1
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline
# plt.bar(genre.keys() , genre.values())
# plt.xticks(rotation='vertical')
# One hot encoding based on genres
movie[list(genre.keys())] = np.nan
i=0
for item in movie['genres']:
    genres = item.split('|')
    movie.loc[i,genres] = 1
    i=i+1
movie.loc[0,8:] = np.nan
movie.fillna(0 , inplace = True)
movie.head()

In [ ]:
rating = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
rating.head()

In [ ]:
movie_avg_ratings = rating.groupby('movieId')['rating'].mean()
movie_avg_ratings.head()

In [ ]:
movie_rating_count = rating.groupby('movieId')['rating'].count()
movie_rating_count

In [ ]:
df_final = pd.merge(movie , movie_avg_ratings, left_on = 'movieId', right_index = True ,  how = 'left')
df_final = pd.merge(df_final , movie_rating_count, left_on = 'movieId', right_index = True ,  how = 'left')
df_final.rename(columns = {'rating_x' : 'avg_rating' , 'rating_y' : 'rating_count'} , inplace=True)
df_final.drop(['genres' , 'movieId'] , axis =1 , inplace = True)
df_final.set_index('title' , inplace=True)
df_final.head()

In [ ]:
df_final.dropna(inplace=True)

In [ ]:
# from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df_final[['rating_count']] = ss.fit_transform(df_final[['rating_count']])

In [ ]:
# genome_scores = pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_scores.csv')
# genome_tags = pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_tags.csv')
# link = pd.read_csv('/kaggle/input/movielens-20m-dataset/link.csv')
# tag = pd.read_csv('/kaggle/input/movielens-20m-dataset/tag.csv')

In [ ]:
from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(df_final)

In [ ]:
query_index = np.random.choice(df_final.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(df_final.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(df_final.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, df_final.index[indices.flatten()[i]], distances.flatten()[i]))